In [1]:
import os
import pandas as pd


# Change work directory to /artifact
print(os.getcwd())
os.chdir("../data/downloads")
print(os.getcwd())

/home/cory/PycharmProjects/bachelor_2022/artifact/data_fetch
/home/cory/PycharmProjects/bachelor_2022/artifact/data/downloads


In [3]:
ds_name = 'ml-100k'
!wget https://files.grouplens.org/datasets/movielens/{ds_name}.zip
!unzip {ds_name}.zip
!rm {ds_name}.zip

In [4]:
# Instantiate all needed dataframes from download files

# Instantiate DF of user-movie rating
data_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data_df = pd.read_csv(f'{ds_name}/u.data', sep='\t', names=data_cols)
print(data_df.sample(1))

# Instantiate DF of user demography
user_cols = ['user_id', 'raw_user_age', 'user_gender', 'user_occupation_text', 'user_zip_code']
user_df = pd.read_csv(f'{ds_name}/u.user', sep='|', names=user_cols)
print(user_df.sample(1))

movie_cols = ['movie_id', 'movie_name', 'release_date', ' ','link', 'genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10', 'genre11', 'genre12', 'genre13', 'genre14', 'genre15', 'genre16', 'genre17', 'genre18', 'genre19', 'genre20']
movie_df = pd.read_csv(f'{ds_name}/u.item', sep='|', names=movie_cols, encoding = "ISO-8859-1")
# keep columns movie_id, movie_name, release_date
movie_df = movie_df.iloc[:, :3]
movie_df.sample(1)

       user_id  movie_id  rating  timestamp
29135      266       275       5  892257831
     user_id  raw_user_age user_gender user_occupation_text user_zip_code
586      587            26           M                other         14216


,movie_id,movie_name,release_date
201,202,Groundhog Day (1993),01-Jan-1993


In [ ]:
# Merge individual dfs into one

# Augment data_df with user_df
df = data_df.merge(user_df, on='user_id')
# Augment df with movie_df
df = df.merge(movie_df, on='movie_id')
df

In [ ]:
# Simple data augmentation

# Bucketize user age (as seen in tfds)
def bucketizer(age):
    if age < 18:
        return 1
    elif age >= 18 and age < 25:
        return 18
    elif age >= 25 and age < 35:
        return 25
    elif age >= 35 and age < 45:
        return 35
    elif age >= 45 and age < 50:
        return 45
    elif age >= 50 and age < 55:
        return 50
    else:
        return 56

df['bucketized_user_age'] = df['raw_user_age'].apply(bucketizer)

# Convert gender to bool value
df['bool_user_gender'] = df['user_gender'].map({'M': True, 'F': False})
df

# Extract year as int from release date
# def year_extractor(date):
#     return date.split('-')[2]
#
# df['release_year'] = df['release_date'].apply(year_extractor)
# df

In [ ]:
os.chdir("../recommender-dataset/")
df.to_csv(f"{ds_name}_augmented.csv", index=False)